# Exploring HEM possibilities

In [1]:
using Revise
using LinearAlgebra
using JuMP
using GLPK
using XLSX
#using Complementarity #not sure I need this one?
#using PATHSolver
using Revise #not having to restart sessions
#using Pandas #optional (only for EXIOBASE data)

include("../src/SUT_structure.jl") # Used for the SUT setup <sut = SUT.structure(...)>
include("../src/Constructs.jl") # Used to derive single-production systems from the SUT setup, e.g. as <itc = Constructs.ITC(sut)>
include("../src/Auxiliary.jl") # Includes some helper functions
include("../src/Model_data.jl") # Sets up the data structure for RCOT modelling based on SUT.structure or Constructs.construct
include("../src/RCOT_model.jl"); # Builds and solves the RCOT models

In [2]:
# Load the workbook
xf = XLSX.readxlsx("../data/SUT_c=i.xlsx")

XLSXFile("../data/SUT_c=i.xlsx") containing 4 Worksheets
            sheetname size          range        
-------------------------------------------------
                    V 4x4           A1:D4        
                  U+e 4x5           A1:E4        
                    F 5x4           A1:D5        
                   pi 5x2           A1:B5        


In [3]:
# Get data from all worksheets and convert it to float64 data type
V = convert(Matrix{Float64}, xf["V!B2:D4"])
U = convert(Matrix{Float64}, xf["U+e!B2:D4"])
e = convert(Matrix{Float64}, xf["U+e!E2:E4"])
F = convert(Matrix{Float64}, xf["F!B2:D5"])
pii = convert(Matrix{Float64}, xf["pi!B2:B5"])
t = xf["V!A1"]

"mixed"

In [4]:
sut = SUT.structure(U,V,F,e,pii,t);

[ Info: The SUT system is not given in monetary units such that no total industry output and dependent matrices may be calculated.
[ Info: A structure for supply-use elements was set up. Changing individual elements will not change others automatically.


In [5]:
model_data = Model_data.SU(sut);

[ Info: You are setting up an SU model dataset. Elements of this dataset are now treated independently, meaning that no recalculation whatsoever takes place when individual elements are changed.


In [6]:
ctc = Constructs.CTC(sut)

construct([113.0; 74.0; 127.0;;], [26.0; 20.0; 68.0;;], [16.74074074074074 -4.611620795107033 74.87088005436628; 53.01234567901234 -6.7655453618756365 7.753199682863292; 0.0 59.0 0.0], [0.14814814814814814 -0.06231919993387883 0.5895344886170574; 0.4691358024691358 -0.0914262886739951 0.06104881640049836; 0.0 0.7972972972972974 0.0], [1.496064824981334 0.5849598137818962 0.9176929158065789; 0.6730818217752205 1.2221748869076374 0.4714172778778162; 0.5366463173613245 0.9744367341560893 1.3758597215512318], [13.950617283950617 12.612640163098877 7.436742552950504; 5.580246913580247 -4.760448521916412 24.180201608336162; 20.925925925925924 6.152905198776758 -1.078831124702684; 2.7901234567901234 12.372069317023445 13.83780722618643], [0.12345679012345678 0.17044108328511998 0.058557027975988225; 0.04938271604938271 -0.06433038543130287 0.1903952882546155; 0.18518518518518517 0.08314736755103729 -0.008494733265375467; 0.024691358024691357 0.16719012590572224 0.108959112017216], [34.0; 24.9

In [9]:
io_data = Model_data.IO(ctc)

[ Info: You are setting up an IO model dataset. Elements of this dataset are now treated independently, meaning that no recalculation whatsoever takes place when individual elements are changed.


construct([113.0; 74.0; 127.0;;], [26.0; 20.0; 68.0;;], [16.74074074074074 -4.611620795107033 74.87088005436628; 53.01234567901234 -6.7655453618756365 7.753199682863292; 0.0 59.0 0.0], [0.14814814814814814 -0.06231919993387883 0.5895344886170574; 0.4691358024691358 -0.0914262886739951 0.06104881640049836; 0.0 0.7972972972972974 0.0], [1.496064824981334 0.5849598137818962 0.9176929158065789; 0.6730818217752205 1.2221748869076374 0.4714172778778162; 0.5366463173613245 0.9744367341560893 1.3758597215512318], [13.950617283950617 12.612640163098877 7.436742552950504; 5.580246913580247 -4.760448521916412 24.180201608336162; 20.925925925925924 6.152905198776758 -1.078831124702684; 2.7901234567901234 12.372069317023445 13.83780722618643], [0.12345679012345678 0.17044108328511998 0.058557027975988225; 0.04938271604938271 -0.06433038543130287 0.1903952882546155; 0.18518518518518517 0.08314736755103729 -0.008494733265375467; 0.024691358024691357 0.16719012590572224 0.108959112017216], [34.0; 24.9

In [10]:
io_data.A

3×3 Matrix{Float64}:
 0.148148  -0.0623192  0.589534
 0.469136  -0.0914263  0.0610488
 0.0        0.797297   0.0

In [22]:
A_star = [ctc.A[1,:]'; zeros(1,size(ctc.A,2)); ctc.A[3,:]']

3×3 Matrix{Float64}:
 0.148148  -0.0623192  0.589534
 0.0        0.0        0.0
 0.0        0.797297   0.0

In [ ]:
# Trying it a different way with a mask
idx_r_extr = [2:3]
idx_c_extr = [1:1]

A_star = deepcopy(ctc.A)

A_star[idx_r_extr, idx_c_extr] .= 0
A_knot = ctc.A - A_star

In [25]:
A_knot = ctc.A[2,:]

3-element Vector{Float64}:
  0.4691358024691358
 -0.0914262886739951
  0.06104881640049836

In [32]:
A_star = ctc.A[2,:] - A_knot

3-element Vector{Float64}:
 0.0
 0.0
 0.0

In [28]:
ctc.A

3×3 Matrix{Float64}:
 0.148148  -0.0623192  0.589534
 0.469136  -0.0914263  0.0610488
 0.0        0.797297   0.0

In [29]:
A_knot

3-element Vector{Float64}:
  0.4691358024691358
 -0.0914262886739951
  0.06104881640049836

In [ ]:
# Load the workbook
xf = XLSX.readxlsx("../data/SUT_c+i-.xlsx")

XLSXFile("../data/SUT_c+i-.xlsx") containing 4 Worksheets
            sheetname size          range        
-------------------------------------------------
                    V 4x6           A1:F4        
                  U+e 6x5           A1:E6        
                    F 5x4           A1:D5        
                   pi 5x2           A1:B5        


In [ ]:
# Get data from all worksheets and convert it to float64 data type
V = convert(Matrix{Float64}, xf["V!B2:F4"])
U = convert(Matrix{Float64}, xf["U+e!B2:D6"])
e = convert(Matrix{Float64}, xf["U+e!E2:E6"])
F = convert(Matrix{Float64}, xf["F!B2:D5"])
pii = convert(Matrix{Float64}, xf["pi!B2:B5"])
t = xf["V!A1"];